<a href="https://colab.research.google.com/github/wesleycoutinhodev/pln-b2w-reviews/blob/main/notebooks/Operacoes_1_2_3_Tratamento_Dados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Notebook: Operacoes_1_2_3_Tratamento_Dados.ipynb
# Seções:
# 1. Setup e Imports
# 2. Carregamento do Dataset
# 3. Operação 1: Análise Exploratória e Tratamento
# 4. Operação 2: Seleção de Colunas Samsung
# 5. Operação 3: Análise de Inconsistências
# 6. Salvamento dos Dados Processados
# 7. Relatório de Execução

In [1]:
# Notebook: Operacoes_1_2_3_Tratamento_Dados.ipynb
# ==================================================

# 1. SETUP INICIAL
from google.colab import drive
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Mount Drive
drive.mount('/content/drive')
BASE_PATH = '/content/drive/MyDrive/PLN_B2W_Reviews'

# Criar estrutura se não existir
import os
for folder in ['data/raw', 'data/processed', 'results/analysis']:
    os.makedirs(f"{BASE_PATH}/{folder}", exist_ok=True)

Mounted at /content/drive


In [4]:
# 2. DOWNLOAD DO DATASET
!wget -O /content/drive/MyDrive/PLN/PLN_B2W_Reviews/data/raw/B2W-Reviews01.csv
BASE_PATH = "/content/drive/MyDrive/PLN/PLN_B2W_Reviews"
# 3. CARREGAMENTO E PRIMEIRA ANÁLISE
df = pd.read_csv(f'{BASE_PATH}/data/raw/B2W-Reviews01.csv')
print(f"📊 Dataset carregado: {df.shape[0]:,} linhas x {df.shape[1]} colunas")
print(f"📋 Colunas: {list(df.columns)}")


df.info()

wget: missing URL
Usage: wget [OPTION]... [URL]...

Try `wget --help' for more options.
📊 Dataset carregado: 132,373 linhas x 14 colunas
📋 Colunas: ['submission_date', 'reviewer_id', 'product_id', 'product_name', 'product_brand', 'site_category_lv1', 'site_category_lv2', 'review_title', 'overall_rating', 'recommend_to_a_friend', 'review_text', 'reviewer_birth_year', 'reviewer_gender', 'reviewer_state']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 132373 entries, 0 to 132372
Data columns (total 14 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   submission_date        132373 non-null  object 
 1   reviewer_id            132373 non-null  object 
 2   product_id             132373 non-null  object 
 3   product_name           132289 non-null  object 
 4   product_brand          40982 non-null   object 
 5   site_category_lv1      132367 non-null  object 
 6   site_category_lv2      128360 non-null  object 
 7   r

In [6]:
# --- Análise e Tratamento do Dataset ---
print("\n--- 3. Operação 1: Análise Exploratória e Tratamento em Todos os Campos ---")

# Verificação de valores ausentes (NaN) em cada coluna
print("\n📊 Valores Ausentes (NaN) por Coluna:")
nan_counts = df.isnull().sum()
nan_counts_df = pd.DataFrame(nan_counts, columns=['NaN Count'])
nan_counts_df['Percentage (%)'] = (nan_counts / len(df)) * 100
print(nan_counts_df)

# Remover a coluna 'review_text_translated' por possuir muitos valores NaN e não ser relevante para a tarefa.
if 'review_text_translated' in df.columns:
    df = df.drop(columns=['review_text_translated'])
    print("\n✅ Coluna 'review_text_translated' removida.")

# Análise de valores inválidos ou inconsistentes em colunas específicas
print("\n🔍 Verificação de Valores Inválidos ou Inconsistentes por Coluna:")

# 1. review_id: Verificar se há IDs duplicados, pois cada ID deve ser único.
if df['reviewer_id'].nunique() < len(df['reviewer_id']):
    print(f"⚠️ Atenção: A coluna 'reviewer_id' contém {len(df['reviewer_id']) - df['reviewer_id'].nunique()} IDs duplicados. Isso pode indicar inconsistência nos dados.")
    # Dependendo da análise, você pode querer remover duplicatas ou investigar mais a fundo.

# 2. product_id: Verificar se há valores ausentes.
if df['product_id'].isnull().sum() > 0:
    print(f"⚠️ Atenção: A coluna 'product_id' contém valores ausentes. Número de NaN: {df['product_id'].isnull().sum()}")

# 3. overall_rating: Verificar se os valores estão no intervalo de 1 a 5.
invalid_ratings = df[~df['overall_rating'].isin(range(1, 6))]
if not invalid_ratings.empty:
    print(f"⚠️ Atenção: A coluna 'overall_rating' contém {len(invalid_ratings)} valores fora do intervalo 1-5. Primeiros 5 valores inválidos:")
    print(invalid_ratings['overall_rating'].head())

# 4. recommend_to_a_friend: Verificar se os valores são apenas 'Yes' ou 'No'.
unique_recommend = df['recommend_to_a_friend'].unique()
if not set(unique_recommend).issubset({'Yes', 'No', np.nan}):
    print(f"⚠️ Atenção: A coluna 'recommend_to_a_friend' contém valores diferentes de 'Yes', 'No' ou NaN. Valores encontrados: {unique_recommend}")

# 5. reviewer_gender: Verificar se os valores são apenas 'M' ou 'F'.
unique_genders = df['reviewer_gender'].unique()
if not set(unique_genders).issubset({'M', 'F', np.nan}):
    print(f"⚠️ Atenção: A coluna 'reviewer_gender' contém valores diferentes de 'M', 'F' ou NaN. Valores encontrados: {unique_genders}")
    # Preencher valores ausentes para garantir consistência
    df['reviewer_gender'] = df['reviewer_gender'].fillna('Não Informado')
    print("Valores ausentes em 'reviewer_gender' preenchidos com 'Não Informado'.")
    print(f"Novos valores únicos na coluna 'reviewer_gender': {df['reviewer_gender'].unique()}")

# 6. reviewer_birth_year: Verificar se há anos de nascimento inválidos (e.g., no futuro ou muito no passado).
current_year = 2025
invalid_years = df[(df['reviewer_birth_year'] < 1925) | (df['reviewer_birth_year'] > current_year)]
if not invalid_years.empty:
    print(f"⚠️ Atenção: A coluna 'reviewer_birth_year' contém {len(invalid_years)} anos de nascimento inconsistentes (menores que 1925 ou maiores que {current_year}).")
    # Dependendo da política de tratamento, você pode remover ou preencher esses valores.

print("\n✅ Etapa 1: Análise e Tratamento de Dados concluída. Inconsistências identificadas e colunas não relevantes removidas.")

print("\n--- Informações do DataFrame Após o Tratamento ---")
df.info()


--- 3. Operação 1: Análise Exploratória e Tratamento em Todos os Campos ---

📊 Valores Ausentes (NaN) por Coluna:
                       NaN Count  Percentage (%)
submission_date                0        0.000000
reviewer_id                    0        0.000000
product_id                     0        0.000000
product_name                  84        0.063457
product_brand              91391       69.040514
site_category_lv1              6        0.004533
site_category_lv2           4013        3.031585
review_title                 302        0.228143
overall_rating                 0        0.000000
recommend_to_a_friend         18        0.013598
review_text                 3275        2.474069
reviewer_birth_year         5984        4.520559
reviewer_gender                0        0.000000
reviewer_state              3991        3.014965

🔍 Verificação de Valores Inválidos ou Inconsistentes por Coluna:
⚠️ Atenção: A coluna 'reviewer_id' contém 19380 IDs duplicados. Isso pode indicar in

In [7]:
# --- Seleção de Colunas e Filtragem ---
print("\n--- 4. Operação 2: Seleção de Colunas Samsung ---")

# Lista de colunas relevantes, conforme a instrução.
colunas_relevantes = ["review_text", "overall_rating", "recommend_to_a_friend", "product_brand"]

# Filtra o DataFrame para manter apenas as colunas relevantes e a marca 'Samsung'
df_samsung = df[df['product_brand'].str.contains('Samsung', case=False, na=False)][colunas_relevantes].copy()

# Remove a coluna 'product_brand' após a filtragem, pois não é mais necessária
df_samsung = df_samsung.drop(columns=['product_brand'])

print(f"\n📊 DataFrame com reviews da marca 'Samsung' criado: {df_samsung.shape[0]:,} linhas x {df_samsung.shape[1]} colunas")
print(f"📋 Novas colunas: {list(df_samsung.columns)}")

# Exibe as primeiras 5 linhas para ver o resultado do filtro
print("\nPrimeiras 5 linhas do novo DataFrame (Samsung):")
print(df_samsung.head())


--- 4. Operação 2: Seleção de Colunas Samsung ---

📊 DataFrame com reviews da marca 'Samsung' criado: 6,015 linhas x 3 colunas
📋 Novas colunas: ['review_text', 'overall_rating', 'recommend_to_a_friend']

Primeiras 5 linhas do novo DataFrame (Samsung):
                                          review_text  overall_rating  \
29  Uma tela impecável. Se sua prioridade é tela e...               4   
37  Melhor custo benefício. Exatamente  como anunc...               5   
56  Muito útil, para pesquisas e baixar publicaçõe...               5   
75  A entrega sempre no prazo e muitas vezes até a...               4   
98  Recomendo tanto o produto quanto a loja, produ...               4   

   recommend_to_a_friend  
29                   Yes  
37                   Yes  
56                   Yes  
75                   Yes  
98                   Yes  


In [8]:
# --- Análise de Inconsistências ---
print("\n--- 5. Operação 3: Análise de Inconsistências ---")

# Análise de inconsistências: notas 1 e 2 com recomendação 'Yes'
inconsistencias_ruins = df_samsung[(df_samsung['overall_rating'].isin([1, 2])) & (df_samsung['recommend_to_a_friend'] == 'Yes')]
print(f"\nInconsistências (nota 1 ou 2, mas recomendou 'Yes'): {inconsistencias_ruins.shape[0]} registros")
print(inconsistencias_ruins.head())

# Análise de inconsistências: notas 4 e 5 com recomendação 'No'
inconsistencias_boas = df_samsung[(df_samsung['overall_rating'].isin([4, 5])) & (df_samsung['recommend_to_a_friend'] == 'No')]
print(f"Inconsistências (nota 4 ou 5, mas não recomendou 'No'): {inconsistencias_boas.shape[0]} registros")
print(inconsistencias_boas.head())

# Remoção das inconsistências para garantir a coerência dos dados
# Combina as condições para remover ambos os tipos de inconsistências em uma única operação
df_samsung_clean = df_samsung[~((df_samsung['overall_rating'].isin([1, 2]) & (df_samsung['recommend_to_a_friend'] == 'Yes')) |
                                ((df_samsung['overall_rating'].isin([4, 5]) & (df_samsung['recommend_to_a_friend'] == 'No'))))].copy()

# A coluna 'recommend_to_a_friend' deve ser removida antes de começar a modelagem
# pois a coluna 'overall_rating' já capta a intenção do usuário
# o mesmo review não pode ser representado por duas classes diferentes
# ou seja, se a 'overall_rating' for 4, 'recommend_to_a_friend' deveria ser 'Yes' (o que já foi tratado)

print(f"\n✅ DataFrame após a remoção de inconsistências: {df_samsung_clean.shape[0]:,} linhas")
print("As inconsistências foram removidas para garantir a coerência entre a nota e a recomendação.")
print("\n--- Informações do DataFrame Após a Limpeza ---")
df_samsung_clean.info()


--- 5. Operação 3: Análise de Inconsistências ---

Inconsistências (nota 1 ou 2, mas recomendou 'Yes'): 86 registros
                                             review_text  overall_rating  \
873    O celular é bom, porém tem muita interferência...               2   
5358   Produto bem embalado e chegou antes do prazo, ...               1   
6601   Este é o terceiro Samsung Galaxy que eu compro...               2   
11405  Bom dia! A bateria descarrega rápida e o tecla...               2   
13942  Efetuei a compra desse oculos pekosite da Amer...               1   

      recommend_to_a_friend  
873                     Yes  
5358                    Yes  
6601                    Yes  
11405                   Yes  
13942                   Yes  
Inconsistências (nota 4 ou 5, mas não recomendou 'No'): 66 registros
                                            review_text  overall_rating  \
432   O produto é excelente porém o aparelho que rec...               5   
1772  Maravilhoso não prec